# Libraries

In [10]:
# standard
import pandas as pd
import numpy as np
from tqdm import tqdm
import time

# reading data
from os.path import basename
import json
from collections import defaultdict
import glob
import re

# visuals
import matplotlib.pyplot as plt
import seaborn as sns

# Read in Results

In [16]:
# Define the pattern to match the filenames
pattern = "df_(.*?)_(.*?)_emb(.*?)_(.*?).csv"

# Use glob to find all csv files matching the pattern
files = glob.glob('../data/Results/df_*_*_emb*_*.csv')

# Initialize a dictionary to hold the data frames
data_frames = {}

for file in files:
    # Extract the basename of the file for regex matching
    file_basename = basename(file)
    
    # Extract model name, embedding length, and loss function using regex
    match = re.match(pattern, file_basename)
    if match:
        result_type = match.group(1)
        model_name = match.group(2)
        embedding_length = match.group(3)
        loss_function = match.group(4)

        # Read the csv file into a pandas DataFrame
        df = pd.read_csv(file)

        # Store the DataFrame in the dictionary with a tuple key for easy sorting
        data_frames[(result_type, model_name, int(embedding_length), loss_function)] = df
    else:
        # If no match, print the filename for review
        print(f"No match for file: {file_basename}")

# Now, you can sort the data frames based on the tuple keys
sorted_keys = sorted(data_frames.keys(), key=lambda x: (x[0], x[1], x[2], x[3]))

# Accessing a sorted DataFrame
for key in sorted_keys:
    print(f"Type: {key[0]} \n Model: {key[1]}, Embedding Length: {key[2]}, Loss Function: {key[3]}")

Type: SystemUsage 
 Model: Linear, Embedding Length: 64, Loss Function: CRPS
Type: SystemUsage 
 Model: eFormer, Embedding Length: 64, Loss Function: CRPS
Type: epoch 
 Model: Linear, Embedding Length: 64, Loss Function: CRPS
Type: epoch 
 Model: eFormer, Embedding Length: 64, Loss Function: CRPS
Type: eval 
 Model: Linear, Embedding Length: 64, Loss Function: CRPS
Type: eval 
 Model: eFormer, Embedding Length: 64, Loss Function: CRPS
